<a href="https://colab.research.google.com/github/calerovillalobos10/WebScraping_Python/blob/main/Lab1_PF3347.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lab1 Análisis en Ciencia de Datos


## Estudiantes

*   Bryan Thomas Calero Villalobos
*   Daniela Montero Parkinson
*   Elemento de la lista

## ¿Qué es web scraping?

### Definición de web scraping

Según Datademia (s.f.) el web scraping se puede ver como el proceso de recolectar datos desde un sitio web, sea de manera manual o automática. De manera similar, Cloudflare (s.f.) define este técnica como el uso de un programa informático para extraer datos que fueron realizados por otro programa.Este método en escencia convierte los datos no estructurados o semiestructurados de páginas web para ser manipulados por herramientas con el fin de ser analizados.


### Ejemplos de aplicaciones de web scraping en el mundo real

Realizando una combinación de lo mencionado por Liu (2021) y Mitchell (2024) algunas herramientas o aplicaciones utilizadas para hacer web scraping son:

*   *ScraperAPI*: esta herramienta tiene integración con Python y NodeJS. Además, facilita el scraping de datos de tablas HTML.
*   *Octoparse*: no se require habilidades de programación para utilizar la herramienta y es efeciva con sitios como: ecommerce, inversiones, criptomonedas y marketing.
*   *Parsehub*: está dirigida a analistas de datos, comercializadores, etc. Es un aplicativo visual de web scraping, en donde se pueden extraer los datos dando click en el sitio web.
*   *Crawly*: no necesita saber de programación, pero extrae elementos limitados. Además, convierte los datos a CSV o JSON.
*   *Scrapy*: está enfocado a desarrolladores de Python y con conocimiento de scraping. Utiliza una biblioteca asincrónica que le permite avanzar con las siguientes tareas antes de finalizar.
*   *Import.io*: esta herramienta extrae información de una web y los organiza por conjunto de datos.



### Herramientas comunes para realizar web scraping en Python

### Consideraciones éticas y legales al realizar web scraping

## Referencias

BeautifulSoup:Richardson, L. (2021). Beautiful Soup Documentation. Recuperado de https://www.crummy.com/software/BeautifulSoup/bs4/doc/

Cloudflare. (s.f.). *¿Qué es el scraping de datos?* https://www.cloudflare.com/es-es/learning/bots/what-is-data-scraping/

Datademia. (s.f.). *¿Qué es el web scraping?* https://datademia.es/blog/que-es-web-scraping

https://rcs.cic.ipn.mx/2015_95/Metodologias%20para%20analisis%20politico%20utilizando%20Web%20Scraping.pdf

Mitchell, A. (12 de Agosto 2024). Herramientas de Web Scraping. Guru99. https://www.guru99.com/es/web-scraping-tools.html

Requests:Reitz, K. (2021). Requests: HTTP for Humans. Recuperado de https://docs.python-requests.org/en/latest/

Scrapy:Scrapy Project. (2024). Scrapy Documentation. Recuperado de https://docs.scrapy.org/en/latest/

Liu, V. (1 de septiembre 2021). Los 30 mejores software gratuitos de web scraping en 2021. Medium. https://medium.com/@veronicaliuoctoparse/los-30-mejores-software-gratuitos-de-web-scraping-en-2021-a9c89d819e03



# Uso práctico - Web Scraping en Gollo

## Extracción de datos

Primero se instalarán las dependencias

*   Se utilizará la librería **requests** para realizar la petición htpp y poder leer el documento HTML de la url brindada *https://www.gollo.com/*
*   Se utilizará la librería **BeautifulSoup** para realizar las extracciones de las diferentes partes de interés

In [ ]:
!pip3 install requests
!pip3 install beautifulsoup4

Se procede a importar las dependencias para poder hacer uso de las mismas

In [ ]:
import requests
from bs4 import BeautifulSoup

Se inicializan y definen las variables a utilizar. En donde la variable *page_number* y *max_pages* se utilizan para controlar el ciclo de la paginación y *all_data* se utiliza para almacenar los datos recolectados

In [ ]:
page_number = 1 # Número de página inicial
max_pages = 130  # Establece un límite máximo para evitar bucles infinitos
all_data = [] # Lista para almacenar todos los datos recolectados

1. Se crea un bucle para recorrer cada página del sitio web. Se construye dinámicamente la URL para cada página y se realiza una solicitud HTTP con *requests.get(url)*. Si la solicitud no es exitosa (código 200) el bucle se detiene


```
    while page_number <= max_pages:

      # Construcción dinámica de la URL para cada página
      url = f'https://www.gollo.com/c?p={page_number}'
      page = requests.get(url)

      # Verificación del estado de la solicitud HTTP
      if page.status_code != 200:
        print(f"Error al acceder a la página {page_number}. Código de estado: {page.status_code}")
        break
```

2. En la siguiente porción de código se utiliza *BeautifulSoup* para analizar el HTML del sitio web. Luego se buscan los bloques que contienen los detalles de los productos y sino se encuentran productos se rompe el ciclo

  * La línea *soup = BeautifulSoup(page.text, 'html.parser')* convierte el contenido HTML del sitio web en un objeto de BeautifulSoup que facilita la búsqueda, navegación y manipulación del HTML en Python. Esto permite extraer y trabajar con elementos específicos del documento de manera sencilla
  * La línea *block_items = soup.find_all(class_='product-item-details')* busca y devuelve todos los elementos HTML del sitio que tienen la clase *product-item-details*. Usa el método *find_all()* de BeautifulSoup, que encuentra todos los elementos que coinciden con ese criterio, y los almacena en la lista block_items para su posterior análisis



```
      # Parseo del contenido HTML de la página
      soup = BeautifulSoup(page.text, 'html.parser')

      # Búsqueda de los bloques que contienen detalles de los productos
      block_items = soup.find_all(class_='product-item-details')

      # Si no hay más productos en la página, se detiene el bucle
      if not block_items:
        print(f"No se encontraron más productos en la página {page_number}.")
        break
```



3. Se extrae y almacena los datos de cada producto en la página actual, en donde se recolecta el nombre del producto, varios precios (final, antiguo y especial) y la categoría. Esto con el fin de almacenarlo en una lista para después poner la información en un dataframe

  * El método *find()* de BeautifulSoup busca y devuelve el primer elemento HTML que coincide con los criterios especificados. En donde:
    *   *block_final_price = items.find(class_='price-final_price')*: Busca el primer elemento dentro de items que tenga la clase *price-final_price*
    * *final_price = block_final_price.find(class_='price').text.strip() if block_final_price else None*: Si se encontró *block_final_price*, busca dentro de él el primer elemento con la clase *price* y extrae el texto. Este proceso se repite para block_old_price* y *block_special_price*, buscando elementos con las clases *old-price* y *special-price*, respectivamente



```
      # Recolección de los datos de cada producto en la página actual
      page_data = []
      for items in block_items:
        # Extracción del nombre del producto
        name = items.find(class_='product-item-link').text.strip()

        # Extracción de precios finales, antiguos y especiales si están disponibles
        block_final_price = items.find(class_='price-final_price')
        final_price = block_final_price.find(class_='price').text.strip() if block_final_price else None

        block_old_price = items.find(class_='old-price')
        old_price = block_old_price.find(class_='price').text.strip() if block_old_price else None

        block_special_price = items.find(class_='special-price')
        special_price = block_special_price.find(class_='price').text.strip() if block_special_price else None

        # Agregar los datos recolectados a la lista de la página actual
        page_data.append({
            "name": name,
            "final_price": final_price,
            "old_price": old_price,
            "special_price": special_price,
            "category": name.split()[0],
        })

      # Almacena los datos de la página actual en la lista principal
      all_data.extend(page_data)
```



In [ ]:
while page_number <= max_pages:

  # Construcción dinámica de la URL para cada página
  url = f'https://www.gollo.com/c?p={page_number}'
  page = requests.get(url)

  # Verificación del estado de la solicitud HTTP
  if page.status_code != 200:
    print(f"Error al acceder a la página {page_number}. Código de estado: {page.status_code}")
    break

  # Parseo del contenido HTML de la página
  soup = BeautifulSoup(page.text, 'html.parser')

  # Búsqueda de los bloques que contienen detalles de los productos
  block_items = soup.find_all(class_='product-item-details')

  # Si no hay más productos en la página, se detiene el bucle
  if not block_items:
    print(f"No se encontraron más productos en la página {page_number}.")
    break

  # Recolección de los datos de cada producto en la página actual
  page_data = []
  for items in block_items:
    # Extracción del nombre del producto
    name = items.find(class_='product-item-link').text.strip()

    # Extracción de precios finales, antiguos y especiales si están disponibles
    block_final_price = items.find(class_='price-final_price')
    final_price = block_final_price.find(class_='price').text.strip() if block_final_price else None

    block_old_price = items.find(class_='old-price')
    old_price = block_old_price.find(class_='price').text.strip() if block_old_price else None

    block_special_price = items.find(class_='special-price')
    special_price = block_special_price.find(class_='price').text.strip() if block_special_price else None

    # Agregar los datos recolectados a la lista de la página actual
    page_data.append({
        "name": name,
        "final_price": final_price,
        "old_price": old_price,
        "special_price": special_price,
        "category": name.split()[0],
    })

  # Almacena los datos de la página actual en la lista principal
  all_data.extend(page_data)

  # Avanza a la siguiente página
  page_number += 1

## Procesamiento de datos

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame(all_data, columns=["name", "final_price", "old_price", "special_price", "category"])
print(df)

                                                  name final_price old_price  \
0        Base de Colchón Herrera Matrimonial Valeriana    ₡259.000      None   
1     Parlante JBL 90W Bluetooth Negro JBLAUTH200BLKAM    ₡199.900  ₡199.900   
2           Celular Nokia 128MB 2.4" Midnight Blue 110     ₡29.900      None   
3         Televisor JVC Google LED 58" UHD  LT-58KB538    ₡309.900  ₡309.900   
4              Base de Colchón Vargas Individual Rombo    ₡159.900      None   
...                                                ...         ...       ...   
3095           Colchón Sealy Matrimonial Hybrid Silver    ₡359.000      None   
3096               Ropero Commodity Aracaju MDP Blanco    ₡379.900      None   
3097             Ropero Herrera Colonial Madera Tabaco    ₡265.000      None   
3098              Ropero Herrera Clásico Madera Tabaco    ₡315.000  ₡315.000   
3099     Ropero Herrera Chifonier Madera de pino Negro    ₡345.000      None   

     special_price   category  
0      

## Visualización de datos